# Knowledge Graph RAG with LlamaIndex, querying Philadelphia Phillies

## Table of Contents

## Installation and Configuration

In [1]:
%pip install llama_index==0.8.31 ipython-ngql nebula3-python pyvis networkx youtube_transcript_api

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import logging
import sys

os.environ["OPENAI_API_KEY"] = "sk-#################"

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

## Connect to Nebula Graph and set up new space

In [3]:
os.environ["GRAPHD_HOST"] = "127.0.0.1"
os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula" 
os.environ["NEBULA_ADDRESS"] = "127.0.0.1:9669"  

%reload_ext ngql
connection_string = f"--address {os.environ['GRAPHD_HOST']} --port 9669 --user root --password {os.environ['NEBULA_PASSWORD']}"
%ngql {connection_string}

Connection Pool Created


,Name
0,demo_basketballplayer
1,phillies_rag


In [4]:
%ngql CREATE SPACE IF NOT EXISTS phillies_rag(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

""


In [5]:
%ngql SHOW SPACES;

,Name
0,demo_basketballplayer
1,phillies_rag


In [6]:
%%ngql
USE phillies_rag;
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);

""


In [7]:
%ngql CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

""


In [8]:
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

space_name = "phillies_rag"
edge_types, rel_prop_names = ["relationship"], ["relationship"]
tags = ["entity"]

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

## Load data and create KG index

In [9]:
from llama_index import (
    LLMPredictor,
    ServiceContext,
    KnowledgeGraphIndex,
)
from llama_index.graph_stores import SimpleGraphStore
from llama_index import download_loader
from llama_index.llms import OpenAI

# define LLM
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

In [10]:
from llama_index import load_index_from_storage
from llama_hub.youtube_transcript import YoutubeTranscriptReader

try:

    storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
    kg_index = load_index_from_storage(
        storage_context=storage_context,
        service_context=service_context,
        max_triplets_per_chunk=15,
        space_name=space_name,
        edge_types=edge_types,
        rel_prop_names=rel_prop_names,
        tags=tags,
        verbose=True,
    )
    index_loaded = True
except:
    index_loaded = False

if not index_loaded:
    
    WikipediaReader = download_loader("WikipediaReader")
    loader = WikipediaReader()
    wiki_documents = loader.load_data(pages=['Philadelphia Phillies'], auto_suggest=False)
    print(f'Loaded {len(wiki_documents)} documents')

    youtube_loader = YoutubeTranscriptReader()
    youtube_documents = youtube_loader.load_data(ytlinks=['https://www.youtube.com/watch?v=k-HTQ8T7oVw'])    
    print(f'Loaded {len(youtube_documents)} YouTube documents')

    kg_index = KnowledgeGraphIndex.from_documents(
        documents=wiki_documents + youtube_documents,
        storage_context=storage_context,
        max_triplets_per_chunk=15,
        service_context=service_context,
        space_name=space_name,
        edge_types=edge_types,
        rel_prop_names=rel_prop_names,
        tags=tags,
        include_embeddings=True,
    )
    
    kg_index.storage_context.persist(persist_dir='./storage_graph')

## Query with Text2Cypher

In [11]:
from llama_index.query_engine import KnowledgeGraphQueryEngine
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from IPython.display import Markdown, display

# query_engine = KnowledgeGraphQueryEngine(
#     storage_context=storage_context,
#     service_context=service_context,
#     llm=llm,
#     verbose=True,
# )

hybrid_query_engine = kg_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=3,
    explore_global_knowledge=True,
)

In [12]:
hybrid_query_engine = kg_index.as_query_engine()
response = hybrid_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a baseball player who was named MVP of the NLCS. He also fueled tensions after signing a 13-year $330 million contract.</b>

In [13]:
query_engine = kg_index.as_query_engine()
response = query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a baseball player who was named MVP of the NLCS. He also fueled tensions after signing a 13-year $330 million contract.</b>

In [14]:
response = query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a baseball player who was named MVP of the NLCS. He also fueled tensions after signing a 13-year $330 million contract.</b>

In [15]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Bryce Harper'
RETURN p, e, m;

,p,e,m
0,"(""Bryce Harper"" :entity{name: ""Bryce Harper""})","(""Bryce Harper"")-[:relationship@45106556397841...","(""13-year $330 million contract"" :entity{name:..."
1,"(""Bryce Harper"" :entity{name: ""Bryce Harper""})","(""Bryce Harper"")-[:relationship@59824214907797...","(""NLCS"" :entity{name: ""NLCS""})"


In [16]:
%ng_draw

<class 'pyvis.network.Network'> |N|=3 |E|=2

In [17]:
response = query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The impact of the standing ovation Trey Turner received seems to have positively influenced his season. Following the standing ovation, Trey Turner had a standout performance, mentioning that the hit he got felt like one of his best swings all year. In the 18 games since the standing ovation, Trey Turner has been excelling with a batting average of .343, five home runs, and 15 RBIs. It appears that the love and support shown by the fans during the standing ovation played a significant role in boosting Trey Turner's confidence and enhancing his on-field performance.</b>

In [18]:
%%ngql 
MATCH (p:`entity`)-[r:`relationship`]->(q:`entity`)
WHERE p.`entity`.`name` == 'Trey Turner' 
RETURN p, r, q;

,p,r,q
0,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-89965735714464...","(""as quick"" :entity{name: ""as quick""})"
1,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-89414636367958...","(""a standing ovation"" :entity{name: ""a standin..."
2,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-89275189104623...","(""in comments"" :entity{name: ""in comments""})"
3,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-86268653017792...","(""25 hits"" :entity{name: ""25 hits""})"
4,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-86268653017792...","(""5 home runs"" :entity{name: ""5 home runs""})"
...,...,...,...
70,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@804341681940123...","(""ball and strike call"" :entity{name: ""ball an..."
71,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@859994387799338...","(""baseball"" :entity{name: ""baseball""})"
72,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@859994387799338...","(""fifth home run"" :entity{name: ""fifth home ru..."
73,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@878913129174356...","(""inside of his own head"" :entity{name: ""insid..."


In [19]:
%ng_draw

<class 'pyvis.network.Network'> |N|=75 |E|=75

In [20]:
response = query_engine.query("Tell me about some of the facts of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The Philadelphia Phillies are a professional baseball team based in Philadelphia. They compete in Major League Baseball as a member of the National League East division. The team has a long history, dating back to their establishment in 1883. They have won two World Series championships, eight National League pennants, and have made 15 playoff appearances. The Phillies have played over 21,000 games and have won over 10,000 of them. They have played at several stadiums in Philadelphia, including Recreation Park, Baker Bowl, Shibe Park (renamed Connie Mack Stadium), Veterans Stadium, and currently, Citizens Bank Park. The team has had both periods of success and periods of futility throughout their history. They have a rivalry with the Washington Nationals, which dates back to the Nationals' original tenure as the Montreal Expos. The Phillies also had a historical rivalry with the Philadelphia Athletics, known as the City Series. The team has retired eight numbers and has a Wall of Fame to honor former players. They have had notable players in their history, including Hall of Famer Mike Schmidt. In recent years, the Phillies have had ups and downs, with periods of success and periods of rebuilding.</b>

In [21]:
%%ngql 
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Philadelphia Phillies' 
RETURN p, e, m;

,p,e,m
0,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@-9215...","(""Joe Girardi"" :entity{name: ""Joe Girardi""})"
1,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@-8815...","(""Major League Baseball"" :entity{name: ""Major ..."
2,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@-4023...","(""15 playoff appearances"" :entity{name: ""15 pl..."
3,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@14545...","(""National League"" :entity{name: ""National Lea..."
4,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@26691...","(""American professional baseball team"" :entity..."
5,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@30666...","(""Lehigh Valley IronPigs"" :entity{name: ""Lehig..."
6,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@42124...","(""Philadelphia"" :entity{name: ""Philadelphia""})"
7,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@91846...","(""eight National League pennants"" :entity{name..."
8,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@91846...","(""two World Series championships"" :entity{name..."


In [22]:
%ng_draw

<class 'pyvis.network.Network'> |N|=10 |E|=9

## Query with vector index

In [23]:
from llama_index import VectorStoreIndex

WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()
wiki_documents = loader.load_data(pages=['Philadelphia Phillies'], auto_suggest=False)
print(f'Loaded {len(wiki_documents)} documents')

youtube_loader = YoutubeTranscriptReader()
youtube_documents = youtube_loader.load_data(ytlinks=['https://www.youtube.com/watch?v=k-HTQ8T7oVw'])    
print(f'Loaded {len(youtube_documents)} YouTube documents')

vector_index = VectorStoreIndex.from_documents(wiki_documents + youtube_documents)
vector_query_engine = vector_index.as_query_engine()

response = vector_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

Loaded 1 documents
Loaded 1 YouTube documents


<b>Bryce Harper is a professional baseball player. He has played for the Philadelphia Phillies since the 2019 season. Harper signed a 13-year, $330 million deal with the Phillies, which was a significant move for the team. In 2021, he won the NL Most Valuable Player Award. Harper has been a key player for the Phillies and has contributed to their success, including their playoff appearances in the 2022 season.</b>

In [24]:
response = vector_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The standing ovation Trey Turner received had a positive impact on his season. After receiving the ovation, Turner had one of his best games in weeks and felt the love and support from the city of Philadelphia. This love and support seemed to have a significant effect on Turner's performance, as he went on to have a batting average of .343, with 25 hits, five home runs, and 15 RBIs in the 18 games following the ovation. The standing ovation and the subsequent success seemed to have boosted Turner's confidence and helped him get out of his slump.</b>

In [25]:
response = vector_query_engine.query("Tell me about some of the facts of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The Philadelphia Phillies are a professional baseball team based in Philadelphia. They compete in Major League Baseball as a member of the National League East division. The team's home stadium is Citizens Bank Park, located in the South Philadelphia Sports Complex. The Phillies have a history dating back to 1883 and have played at several stadiums in the city over the years. They have won two World Series championships, eight National League pennants, and have made 15 playoff appearances. The team has a loyal fan base and is known for its longevity. The Phillies have had both periods of success and periods of futility throughout their history. They were the last of the pre-expansion teams to win a World Series, with their first championship coming in 1980. Since the start of the Divisional Era in 1969, the Phillies have been one of the most successful teams in MLB, winning 11 division titles, eight National League pennants, and two World Series championships. The team's Triple-A affiliate is the Lehigh Valley IronPigs, and their Double-A affiliate is the Reading Fightin Phils. The Phillies' spring training facilities are located in Clearwater, Florida.</b>

## Create CustomRetriever to combine vector store index and KG index

In [26]:
from llama_index import QueryBundle
from llama_index.schema import NodeWithScore
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever
from typing import List

class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [27]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever, KGTableRetriever

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)

## Create 5 query engines and run queries

In [28]:
# KG default query engine
kg_query_engine = kg_index.as_query_engine()

# KG keyword query engine
kg_keyword_query_engine = kg_index.as_query_engine(
    # setting to false uses the raw triplets instead of adding the text from the corresponding nodes
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

# KG hybrid query engine
kg_hybrid_query_engine = kg_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=3,
    explore_global_knowledge=True,
)

# Vector query engine
vector_query_engine = vector_index.as_query_engine()

# Custom combo query engine
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

In [68]:
response = kg_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a baseball player who was named MVP of the NLCS. He also fueled tensions after signing a 13-year $330 million contract.</b>

In [69]:
response = kg_keyword_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a person who has been involved in various relationships and events. He was named MVP of the NLCS, and he also fueled tensions after signing a 13-year $330 million contract.</b>

In [73]:
response = kg_hybrid_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a star-outfielder who has made a significant impact in the baseball world. He is known for his notable performance and contributions to the Philadelphia Phillies, where he signed a lucrative 13-year $330 million contract as a free agent in 2019. Harper's signing with the Phillies fueled tensions in the rivalry between the Phillies and the Washington Nationals, as he had previously played for the Nationals. Despite Harper's absence from the Nationals in 2019, the team went on to win the World Series that year. Additionally, Harper's achievements include being named the Most Valuable Player (MVP) of the National League Championship Series (NLCS).</b>

In [71]:
response = vector_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a professional baseball player who has played for the Philadelphia Phillies since the 2019 season. He was signed to a 13-year, $330 million deal, making it a significant move for the Phillies. Harper has been a key player for the team and has had notable achievements, including winning the NL Most Valuable Player Award for the 2021 season. He has been a major highlight for the Phillies and has contributed to their success on the field.</b>

In [72]:
response = custom_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a professional baseball player who has played for the Philadelphia Phillies. He joined the team in 2019 after signing a 13-year, $330 million contract, which was a significant move for the Phillies. Harper has been a standout player for the team and has had notable achievements, including winning the NL Most Valuable Player Award in 2021. He has been a key player in the Phillies' recent success, including their playoff appearances in 2022 and their advancement to the World Series. Harper's contributions to the team have made him a highly regarded player in the league.</b>

In [34]:
response = custom_query_engine.query("Tell me some positive facts about Trey Turner.")
display(Markdown(f"<b>{response}</b>"))

<b>Trey Turner has won the World Series and has helped lead his team to the WBC Championship. He is a two-time All-Star and a one-time Silver Slugger. He is known for his speed and has recorded three Cycles, tying the all-time MLB record.</b>

In [64]:
response = kg_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The impact of the standing ovation Trey Turner received on his season seems to have been positive. Following the standing ovation, Trey Turner had a standout performance, describing the hit he got as one of his best swings all year. In the 18 games that followed, Trey Turner has been performing well, boasting a batting average of .343, hitting five home runs, and driving in 15 RBIs. It appears that the love and support shown by the fans during the standing ovation played a role in boosting Trey Turner's confidence and enhancing his on-field performance.</b>

In [65]:
response = kg_keyword_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The standing ovation Trey Turner received likely had a positive impact on his season. It may have boosted his confidence and morale, motivating him to perform better. Additionally, the standing ovation could have served as a recognition and appreciation for his achievements, which could have further motivated him to excel in his performance.</b>

In [66]:
response = kg_hybrid_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The standing ovation Trey Turner received likely had a positive impact on his season. It may have boosted his confidence and morale, leading to improved performance on the field. Additionally, the standing ovation could have served as a motivating factor for Turner, inspiring him to continue working hard and striving for success. Overall, the standing ovation likely had a positive influence on Turner's season.</b>

In [63]:
response = vector_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The standing ovation Trey Turner received had a positive impact on his season. After receiving the ovation, Turner had one of his best games in weeks and felt the love and support from the city of Philadelphia. This love and support seemed to have a significant effect on Turner's performance, as he went on to have a batting average of .343, with 25 hits, five home runs, and 15 RBIs in the 18 games following the ovation. The standing ovation and the subsequent success seemed to boost Turner's confidence and improve his overall performance on the field.</b>

In [67]:
response = custom_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The standing ovation Trey Turner received had both a negative and positive impact on his season. It seemed to initially contribute to his frustrations and struggles, leading to a decline in his performance on both offense and defense. However, the ovation also served as a turning point for Turner. It made him realize that something needed to change, and he took it upon himself to work harder and improve his performance. As a result, his performance significantly improved in the games following the ovation, with a higher batting average, OPS, and several hits, home runs, and RBIs. Overall, the standing ovation seemed to have a transformative effect on Turner's season.</b>

In [40]:
response = custom_query_engine.query("Tell me about some of the facts of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The Philadelphia Phillies are a professional baseball team based in Philadelphia. They compete in Major League Baseball as a member of the National League East division. The team has a long history, being founded in 1883. They have won two World Series championships, eight National League pennants, and have made 15 playoff appearances. The Phillies have played over 21,000 games, with a record of 10,022 wins and 11,187 losses. They have played at several stadiums in Philadelphia, including Recreation Park, Baker Bowl, Shibe Park, Veterans Stadium, and currently Citizens Bank Park. The team has a passionate fan base and a rivalry with the Washington Nationals. The Phillies have had 33 players inducted into the Baseball Hall of Fame.</b>

In [59]:
response = kg_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The current stadium of the Philadelphia Phillies is Citizens Bank Park. It has been the team's home stadium since 2004 and is located in the South Philadelphia Sports Complex. Citizens Bank Park has hosted various events, including Phillies games, concerts, and other sporting events.</b>

In [60]:
response = kg_keyword_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The current stadium of the Philadelphia Phillies is named Veterans Stadium. It was opened in 1971 and used from 1970 to 1991.</b>

In [75]:
response = kg_hybrid_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The current stadium of the Philadelphia Phillies is called Citizens Bank Park. It has been the team's home stadium since 2004 and is located in the South Philadelphia Sports Complex. Citizens Bank Park has hosted various events, including Phillies games, concerts, and other sporting events.</b>

In [58]:
response = vector_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>Citizens Bank Park is the current stadium of the Philadelphia Phillies. It has been their home stadium since 2004 and is located in the South Philadelphia Sports Complex.</b>

In [74]:
response = custom_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The current stadium of the Philadelphia Phillies is Citizens Bank Park. It has been the team's home stadium since 2004 and is located in the South Philadelphia Sports Complex. Citizens Bank Park has a seating capacity of approximately 43,000 and is known for its unique features such as the Liberty Bell and the Ashburn Alley, which pays tribute to former Phillies player Richie Ashburn. The stadium has also undergone upgrades, including a new video display screen in left field, making it the largest in the National League.</b>